# Exploration of the i2b2 data to make sure that concepts don't span multiple lines and if they do, print

In [9]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
%autoreload

import os, random, pandas as pd, numpy as np
import pickle
import sys
import glob
from tqdm import tqdm
import ast
sys.path.append('../../')
# sys.path.append('../ddi_preprocess')
from relation_extraction.data import utils
import nltk
import itertools
from ast import literal_eval # to convert the string tuple form to an actual tuple
RESOURCE_PATH = "/data/medg/misc/geeticka/relation_extraction/i2b2"
#outdir = 'pre-processed/original/'
def res(path): return os.path.join(RESOURCE_PATH, path)

## First look at training data from beth, then partners and then go through the test data

In [11]:
beth_training_concepts = res('concept_assertion_relation_training_data/beth/concept/')

In [12]:
partners_training_concepts = res('concept_assertion_relation_training_data/partners/concept/')

In [13]:
test_concepts = res('reference_standard_for_test_data/concepts/')

In [14]:
def find_span_multiple_lines(directory):
    total_files_to_read = glob.glob(directory + '*.con')
    if not total_files_to_read:
        print("There are no files to read! Check your directory")
    for file in tqdm(total_files_to_read):
        with open(file, 'r') as read_file:
            for line in read_file:
                line = line.strip()
                first_part = line.split('||')[0]
                pos1 = first_part.split(' ')[-2]
                pos2 = first_part.split(' ')[-1]
                line1 = pos1.split(':')[0]
                line2 = pos2.split(':')[0]
                if line1 != line2:
                    print("Concept spans multiple lines", line)

In [15]:
find_span_multiple_lines(beth_training_concepts)

100%|██████████| 73/73 [00:00<00:00, 195.91it/s]


In [16]:
find_span_multiple_lines(partners_training_concepts)

100%|██████████| 97/97 [00:00<00:00, 203.45it/s]


In [17]:
find_span_multiple_lines(test_concepts)

100%|██████████| 256/256 [00:01<00:00, 200.08it/s]


Now, we have confirmed that no concept spans multiple lines. This makes the job easier. 